# Conversion to GeoDataFrame for Visualization with HoloViz Packages
Authors: [Philip Chmielowiec](https://github.com/philipc2), [Ian Franda](https://github.com/ifranda)

## Overview
This notebook showcases the necessary workflow for visualizing Unstructured Grids using UXarary paired with HoloViz. Specifically, outlined is the conversion to a `GeoDataFrame` which allows visualization with packages from the HoloViz stack. Showcased are basic visualization examples using the `hvPlot` pacakge.

In [ ]:
import uxarray as ux
import numpy as np
import warnings

import matplotlib
import matplotlib.pyplot as plt

import hvplot.pandas
import holoviews as hv

warnings.filterwarnings("ignore")

## Data

For this notebook, we will be using E3SM output, {elaborate more on the data}.

In [ ]:
base_path = "../../test/meshfiles/ugrid/outCSne30/"
grid_path = base_path + "outCSne30.ug"
data_path = base_path + "outCSne30_vortex.nc"

In [ ]:
uxds = ux.open_dataset(grid_path, data_path)

In [ ]:
uxds

In [ ]:
uxds.uxgrid

## Conversion to `spatialpandas.GeoDataFrame` for Visualization

In order to support visualization with the popular HoloViz stack of libraries (hvPlot, HoloViews, Datashader, etc.), UXarray provides methods for converting `Grid` and `UxDataArray` objects into a SpatialPandas `GeoDataFrame`, which can be used for visualizing the nodes, edges, and polygons that represent each grid, in addition to data variables.


### `Grid` Conversion

If you wish to *only* represent the grid as geometries without any data variables mapped to them, you can use the `Grid.to_geodataframe()` method to obtain a `GeoDataFrame` with a singular geometry column representing each face represented as a `MultiPolygon`

In [ ]:
gdf_grid = uxds.uxgrid.to_geodataframe()
gdf_grid

### `UxDataArray` & `UxDataset` Conversion

If you are interested in mapping data to each face, you can index the `UxDataset` with the variable of instance (in this case "psi") to return the same `GeoDataFrame` as above, but now with data mapped to each face.

In [ ]:
gdf_data = uxds['psi'].to_geodataframe()
gdf_data

### Challenges with Representing Geoscience Data as Geometries

When we convert to a `GeoDataFrame`, we internally represent the surface of a sphere as a collection of polygons over a 2D projection. However, this leads to issues around the Antimeridian (180 degrees east or west), which polygons are incorrectly constructed and have incorrect geometries. When constructing the `GeoDataFrame`, UXarray detects and corrects any polygon that touches or crosses the antimeridian. An array of indices of these faces can be accessed as part of the `Grid` object.
<br>

<figure>
<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/Earth_map_with_180th_meridian.jpg/640px-Earth_map_with_180th_meridian.jpg" style="height: 300px; width:600px;"/></center>
<center><figcaption>Antimeridian Visual</figcaption></center>
</figure>


In [ ]:
uxds.uxgrid.antimeridian_face_indices

Taking a look at one of these faces that crosses or touches the antimeridian, we can see that it's split across the antimeridian and represented as a `MultiPolygon`, which allows us to properly render this two dimension grid.

In [ ]:
gdf_data.geometry[uxds.uxgrid.antimeridian_face_indices[0]]

For more details about the algorithm used for splitting these polygons, see the [Antimeridian Python Package](https://antimeridian.readthedocs.io/en/stable/).


## Visualizing Geometries

### Nodes

In [ ]:
hv.extension("matplotlib")

plot_kwargs = {"size": 6.0, "xlabel": "Longitude", "ylabel": "Latitude",
               "coastline": True, "width": 1600, "title": "Node Plot (Matplotlib Backend)"}


gdf_grid.hvplot.points(**plot_kwargs)

In [ ]:
hv.extension("bokeh")

plot_kwargs = {"s": 1.0, "xlabel": "Longitude", "ylabel": "Latitude", "coastline": True, "frame_width": 700, "title": "Node Plot (Bokeh Backend)"}

gdf_grid.hvplot.points(**plot_kwargs)

### Edges

In [ ]:
hv.extension("matplotlib")

plot_kwargs = {"linewidth": 1.0, "xlabel":" Longitude", "ylabel": "Latitude", "coastline": True, "width": 1600 , "title": "Edge Plot (Matplotlib Backend)", "color": "black"}

import cartopy.crs as ccrs

gdf_grid.hvplot.paths(**plot_kwargs)

In [ ]:
hv.extension("bokeh")

plot_kwargs = {"line_width": 0.5, "xlabel": "Longitude", "ylabel": "Latitude", "coastline": True, "frame_width": 700, "title": "Edge Plot (Bokeh Backend)"}

gdf_grid.hvplot.paths(**plot_kwargs)

## Visualizing Data Variables

In [ ]:
hv.extension("matplotlib")

plot_kwargs = {"c": "psi", "cmap": "inferno", "width": 400, "height": 200, "title": "Filled Polygon Plot (Matplotlib Backend, Rasterized)", "xlabel":" Longitude", "ylabel": "Latitude"}

gdf_data.hvplot.polygons(**plot_kwargs, rasterize=True)

```{note}
Visualuzing filled polygons without rasterization using the matplotlib backend produces incorrect results, see [hvplot/#1099](https://github.com/holoviz/hvplot/issues/1099)
```

In [ ]:
hv.extension("bokeh")

plot_kwargs = {"c": "psi",  "cmap": "inferno", "line_width": 0.1,  "frame_width": 500, "frame_height": 250, "xlabel":" Longitude", "ylabel": "Latitude"}

gdf_data.hvplot.polygons(**plot_kwargs, rasterize=True)

hv.Layout(gdf_data.hvplot.polygons(**plot_kwargs, rasterize=True).opts(title="Filled Polygon Plot (Bokeh Backend, Rasterized)") +
          gdf_data.hvplot.polygons(**plot_kwargs, rasterize=False).opts(title="Filled Polygon Plot (Bokeh Backend, Vector)")).cols(1)

## Geographic Projections

In [ ]:
hv.extension("matplotlib")

plot_kwargs = {"c": "psi", "cmap": "inferno", "width": 400, "height": 200, "title": "Filled Polygon Plot (Matplotlib Backend, Rasterized)", "xlabel":" Longitude", "ylabel": "Latitude"}

gdf_data.hvplot.polygons(**plot_kwargs, rasterize=True, projection=ccrs.Orthographic())